In [48]:
import os
os.getcwd()

'/home'

In [49]:
os.chdir("..")
import utils.functions as fc
import utils.indicators as indic

In [50]:
from importlib import reload
fc = reload(fc)
indic = reload(indic)

In [51]:
ship_data_enriched = fc.create_ship_data_enriched()

In [52]:
ais = fc.read_ais_parquet()

In [53]:
ais.shape

(4406590, 6)

In [54]:
ais_enriched = fc.enrich_AIS_data(ais, ship_data_enriched)

In [18]:
#ais_enriched = fc.filter_cargo(ais_enriched)

In [11]:
ais_enriched.shape

(2634715, 125)

In [55]:
# function qui assigne le pays d'origine
ais_enriched2, origin_countries = indic.assign_country_origin(ais_enriched)

/home/onyxia/work/hackathon-un-2022/utils/indicators.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  mmsi_number = pd.read_csv(fs.open(f'{BUCKET}/AIS/mmid.csv',


Classification of origin country
Russian Federation                                                                   387
Republic of Türkiye                                                                  158
Panama (Republic of)                                                                 120
Malta                                                                                 70
Liberia (Republic of)                                                                 54
Romania                                                                               47
Marshall Islands (Republic of the)                                                    47
Bulgaria (Republic of)                                                                36
Ukraine                                                                               26
Comoros (Union of the)                                                                21
Sierra Leone                                                                 

In [36]:
import pandas as pd
pd.option_context('display.max_rows', None)

In [57]:
ais_enriched3["mmsi"].nunique()

1177

In [56]:
# fonction that assigns a destination country
ais_enriched3, destination_ports = indic.assign_destination_country(ais_enriched2)

Percentage missing destination port
0.06966864910790145
Number missing destination port
82
mmsi
207008000    [POTI, VARNA, NECOCHEA, CHORNOMORSK, CHERNOMORSK]
207037000    [VARNA, SMEDEREVO, VARNA     ORAGE, JP SGM NAK...
207038000                              [OPENSEA, PP#23, VARNA]
207094000                                               [None]
207095000                                               [None]
                                   ...                        
677075600    [KHERSON/UKRAINE, BANDIRMA/TURKEY, LULA FIELD,...
677080700                  [ROSTOV / RUSSIA, BANDIRMA, RU ROV]
677085700                           [RASA, ALEXANDRIA, UA NIK]
677092400                  [BANGHAZI / LIBYA, 0, JP OIT, None]
677096200                      [MISURATA, NLKAM10084HRARE0001]
Name: destination, Length: 1177, dtype: object
Classification of destination countries
Turkey                                           114
Russia                                            90
Romania           

In [183]:
import numpy as np
ais_enriched3["destination2"] = (ais_enriched3["destination"].replace("None", np.nan)
                        .replace('B 2 A', np.nan)
                        .replace('J0', np.nan)
                        .replace('0', np.nan)
                        .replace(',',np.nan)
                        .replace('G', np.nan)
                        .replace('I0', np.nan)
                        .replace('C', np.nan)
                        .replace('D', np.nan)
                        .replace('C0', np.nan)
                        .replace('S0', np.nan).replace('M', np.nan)
                        .replace('X', np.nan).replace('A', np.nan)
                        .replace('ZRP', np.nan)
                        .replace('AT SEA', np.nan)
                        .replace("$", np.nan)
                        .replace(r'(\s\/\s[a-zA-Z]+)', '', regex=True)
                        .replace(r'(\/[a-zA-Z]+)', '', regex=True)
                        .replace('FISHING AREA', np.nan))

In [70]:
# cleaning up case when destination = port / country
#ais_enriched3["destination2"]  = ais_enriched3["destination2"].replace(r'^SE\s', '')
#test = ais_enriched3["destination2"].str.extract((r'(SE)(\s)'))

In [184]:
ais_enriched3["destination2"].nunique() # 4038

4037

In [185]:
ais_enriched3["destination2"].unique().tolist()

['BILBAO',
 None,
 'ROTTERDAM',
 'KHERSON',
 'SAMSUN',
 'AMSTERDAM',
 'KALAMIS',
 'FUJAIRAH',
 'PDOCAS PORTO RIO',
 'YAN TAI',
 'DUNEDIN',
 'BAKIRKOY',
 'DURBAN',
 'AU NTL',
 nan,
 '> UA YUZ',
 'BUYUKDERE',
 'ISTANBUL',
 'BUYUKDERE         A0',
 'KRYOS',
 'PHV KM ..',
 'AU BUY',
 'SINGAPORE-',
 'BURGAS',
 'AGBAMI,NIGERIA',
 'DILISKELESI:TURKY',
 'HOUSTON FOR ORDERS',
 'SULINA',
 'VARNA BULGARIA',
 'GALATI>ROMANIA',
 'TRIST',
 'YANBU',
 'LA PAMPILLA',
 'AU DPO',
 'TRIST             A0',
 'LA LIBERTAD',
 'HOPA',
 'HEREKE',
 'IGSAS',
 'SEPT ISLES, CANADA',
 'ISKENDERUN',
 'ISKENDERUL',
 'KARABIGA',
 'GUARD POLARCUS ASIMA',
 'ALIAGA',
 'ORDER',
 'YUZHNY',
 'ROGAL',
 'BRAILA',
 'TRGEM',
 'ROSTOV RU ROV',
 'PA PAN',
 'JP NGO OFF',
 'ROSTOV RU',
 'SAMSUN TR SSX',
 'BANDIRMA TR BDM',
 'GEMLIK',
 'KARTAL',
 'SIGRI_GR',
 'TUZLA',
 'IZMIT BAY',
 'TEMRYUK',
 'IZMIT',
 'CONSTANTA',
 'MINDELO',
 'PTPDL',
 'UST LUGA',
 'BRPNG',
 'ES CEU',
 'FIKTK',
 'RUULU',
 'SG SIN',
 'ISTANBUL0',
 'CAPE TOWN, RSA'

In [186]:
ports = pd.read_csv('https://msi.nga.mil/api/publications/download?type=view&key=16920959/SFH00000/UpdatedPub150.csv')
ports = ports.loc[:, ["Main Port Name", "Country Code"]]
ports["Main Port Name"] = ports["Main Port Name"].str.upper()
ports = ports.rename({"Main Port Name" : "destination"}, axis=1)

In [109]:
ports.head()

,destination,Country Code
0,ST. JAMES HARBOR,United States
1,MELCHIOR HARBOR,Antarctica
2,DONGGALA,Indonesia
3,PSVM TERMINAL,Angola
4,KOKOPO,Papua New Guinea


In [111]:
ais_enriched3.shape

(3333279, 129)

In [187]:
ais_enriched4 = ais_enriched3.loc[~ais_enriched3["destination2"].isna(),:]

In [188]:
ais_enriched4.shape

(3076316, 129)

In [189]:
test = ais_enriched4.groupby("mmsi")["destination2"].first() # imparfait car on n'a pas toutes les destinations d'un bateau

In [190]:
test.head()

mmsi
207008000       POTI
207037000      VARNA
207038000    OPENSEA
207133000      VARNA
207228000      VARNA
Name: destination2, dtype: object

In [192]:
test = test.reset_index().copy().iloc[0:100,:]

In [193]:
test.head()

,index,mmsi,destination2
0,0,207008000,POTI
1,1,207037000,VARNA
2,2,207038000,OPENSEA
3,3,207133000,VARNA
4,4,207228000,VARNA


In [194]:
cart_prod = test.merge(ports, how='cross')

In [195]:
ports.head()

,destination,Country Code
0,ST. JAMES HARBOR,United States
1,MELCHIOR HARBOR,Antarctica
2,DONGGALA,Indonesia
3,PSVM TERMINAL,Angola
4,KOKOPO,Papua New Guinea


In [196]:
cart_prod.shape

(288000, 5)

In [197]:
cart_prod[["destination2", "destination"]].head()

,destination2,destination
0,POTI,ST. JAMES HARBOR
1,POTI,MELCHIOR HARBOR
2,POTI,DONGGALA
3,POTI,PSVM TERMINAL
4,POTI,KOKOPO


In [198]:

def check_distance_levenshtein(data: pd.DataFrame, variables=None) -> pd.DataFrame:
    """
    Compute levenshtein distance between list of columns of a Pandas DataFrame
    Parameters
    ----------
    data: Pandas dataframe with some columns to be compared
    variables: List of variables from which distance will be computed. The first one will be used as benchmark
    Returns
    -------
    Initial dataframe with new columns representing levenshtein ratio (computed using :meth:`rapidfuzz.fuzz.partial_ratio`)
    between variables
    """

    if variables is None:
        variables = ['libel_clean_relevanc', 'libel_clean_OFF', 'libel_clean_IRI']

    data[variables] = data[variables].astype(str)
    newvars = ["ratio{}".format(str(i)) for i in range(1, len(variables))]
    data[newvars] = pd.concat(
        [data.apply(lambda x: rapidfuzz.fuzz.partial_ratio(x[variables[0]], x[y]), axis=1) for y in variables[1:]],
        axis=1
    )
    return data


In [200]:
test2 = check_distance_levenshtein(cart_prod, ["destination2", "destination"])

In [202]:
test2.head(2000)

,index,mmsi,destination2,destination,Country Code,ratio1
0,0,207008000,POTI,ST. JAMES HARBOR,United States,33.333333
1,0,207008000,POTI,MELCHIOR HARBOR,Antarctica,33.333333
2,0,207008000,POTI,DONGGALA,Indonesia,33.333333
3,0,207008000,POTI,PSVM TERMINAL,Angola,40.000000
4,0,207008000,POTI,KOKOPO,Papua New Guinea,66.666667
...,...,...,...,...,...,...
1995,0,207008000,POTI,SEKUPANG,Indonesia,25.000000
1996,0,207008000,POTI,TYNEMOUTH,United Kingdom,50.000000
1997,0,207008000,POTI,ASHLAND,United States,0.000000
1998,0,207008000,POTI,CALAIS,France,33.333333


In [204]:
idx = test2.groupby(['destination2'])['ratio1'].transform(max) == test2['ratio1']

In [205]:
test4 = test2[idx]

In [206]:
test4.head(100)

,index,mmsi,destination2,destination,Country Code,ratio1
1447,0,207008000,POTI,POTI,Georgia,100.000000
5150,1,207037000,VARNA,VARNA,Bulgaria,100.000000
7037,2,207038000,OPENSEA,ODENSE,Denmark,83.333333
10910,3,207133000,VARNA,VARNA,Bulgaria,100.000000
13790,4,207228000,VARNA,VARNA,Bulgaria,100.000000
...,...,...,...,...,...,...
127512,44,215135000,BE,AMBES,France,100.000000
127543,44,215135000,BE,PORTO BELO,Mozambique,100.000000
127601,44,215135000,BE,LUBECK,Germany,100.000000
127618,44,215135000,BE,CAMPBELTOWN,United Kingdom,100.000000
